## Load required modules.

TODO: Select according to shape. For example, exclude intrusions.

In [1]:
%matplotlib widget
# Import ncessary modules.
from sklearn.decomposition import PCA
import xlrd
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler


## Read EXCEL data file into data frame.

In [2]:
# File path to your Excel file.
file_path = 'Geochemistry Results-AGG reduced variables.xlsx'

# Define the sheet name to read.
sheet_name = 'Litogeochemistry - clean'

# Read only two columns "Th_ppm" and "U_ppm".
df = pd.read_excel(file_path, sheet_name=sheet_name)
maxzn = df.iloc[:, 63].max()


In [3]:
df.head()

,Sample Description,Year,Depth(meters),Lithology,Shape,Comment,PTS,Al2O3_pct,BaO_pct,CaO_pct,...,Cd_ppm,Co_ppm,Cu_ppm,Li_ppm,Mo_ppm,Ni_ppm,Pb_ppm,Zn_ppm,C_pct,S_pct
0,S10-01,2022,4.44,Di-Tr dolomitic marble,Marble Units,NaN,No,0.73,0.02,26.7,...,0.25,4.0,10,20,1.0,6.0,4,27,7.22,0.23
1,S10-02,2022,14.49,Dolomitic marble,Marble Units,NaN,No,0.68,0.08,30.3,...,0.25,2.0,9,20,1.0,5.0,6,22,9.99,0.75
2,S10-03,2022,15.74,Dolomitic marble,Marble Units,NaN,No,0.56,0.01,29.4,...,0.25,1.0,5,10,1.0,4.0,3,23,10.55,0.58
3,S10-04,2022,26.80,Calcitic marble,Marble Units,NaN,Yes,0.92,0.04,37.0,...,0.25,0.5,4,20,1.0,3.0,10,31,7.76,0.26
4,S10-05,2022,27.02,Calcitic marble,Marble Units,NaN,Yes,3.18,0.08,35.4,...,0.25,3.0,9,10,1.0,7.0,4,13,7.79,1.22


In [4]:
df.shape

(103, 66)

## Remove symbols "<" and "NaN".

In [ ]:
df = df.iloc[: , 7:]
df=df.replace('\<','',regex=True).astype(float)
df = df[np.isfinite(df).all(1)]
df = df.dropna()
df.head()

In [ ]:
df.shape

## Scale data and apply PCA.

In [ ]:
# Scale data before applying PCA
scaling=StandardScaler()
 
# Use fit and transform method 
scaling.fit(df)
Scaled_data=scaling.transform(df)
 
# Set the n_components=3
pca=PCA(n_components=3)
pca.fit(Scaled_data)
x=pca.transform(Scaled_data)
 
# Check the dimensions of data after PCA
print(x.shape)

## Create a datafram for PCA componenets and scale it.

In [ ]:
# Create dataframe
pca_df = pd.DataFrame(
    data=x, 
    #columns=['PC1', 'PC2', 'PC3'])
    columns=['PC'+str(i) for i in range(1, len(pca.components_)+1)])
pca_df

In [ ]:
# Create the scaled PCA dataframe
pca_df_scaled = pca_df.copy()
  
scaler_df = pca_df[['PC1', 'PC2', 'PC3']]
scaler = 1 / (scaler_df.max() - scaler_df.min())
for index in scaler.index:
    pca_df_scaled[index] *= scaler[index]

## Display loadings.

In [ ]:
#loadings = pd.DataFrame(pca.components_.T, columns=['PC1', 'PC2', 'PC3'], index=df.columns)
loadings = pd.DataFrame(pca.components_.T, columns=['PC'+str(i) for i in range(1, len(pca.components_)+1)], index=df.columns)
loadings

## Display sorted loadings.

In [ ]:
sorted_loadings = loadings.sort_values(by='PC1', ascending=False)  # Change 'ascending' as needed
sorted_loadings

In [ ]:
sorted_loadings = loadings.sort_values(by='PC2', ascending=False)  # Change 'ascending' as needed
sorted_loadings

In [ ]:
sorted_loadings = loadings.sort_values(by='PC3', ascending=False)  # Change 'ascending' as needed
sorted_loadings

## Plot a 3D scatter plot with arrows for PCA directions.

In [ ]:
import pandas as pd

# File path to your Excel file.
file_path = 'Geochemistry Results-AGG reduced variables.xlsx'

# Define the sheet name to read.
sheet_name = 'Litogeochemistry - clean'

# Read only the 3rd column.
dc = pd.read_excel(file_path, sheet_name=sheet_name, usecols=[3])

dc = dc.rename(columns={dc.columns[0]: 'Lithology'})

color_map = {
    'Calcitic marble': 'lime',
    'Dolomitic marble': 'blue',
    'Di-Tr dolomitic marble': 'cyan',
    'Carbonatite': 'deeppink',
    'Carbonatite-like': 'pink',
    'Syenite': 'black',
    'Altered Syenite': 'black',
    'Syenite-like': 'black',
    'Impure Siliciclastic': 'grey',
    'Pure Siliciclastic': 'lightgrey'
}

dc['Color'] = dc['Lithology'].map(color_map)
#dc[dc.columns[0]] = dc[dc.columns[0]].map(color_map)

dc=dc.replace('\<','NaN',regex=True)
dc = dc[np.isfinite(df).all(1)]
dc = dc.dropna()

print(dc)
print (pca_df_scaled)
print("Shape of y:", dc.shape)
#print("Shape of X:", X.shape)


In [ ]:
import pandas as pd

# File path to your Excel file.
file_path = 'Geochemistry Results-AGG reduced variables.xlsx'

# Define the sheet name to read.
sheet_name = 'Litogeochemistry - clean'

# Read only the 3rd column.
ds = pd.read_excel(file_path, sheet_name=sheet_name, usecols=[4])

ds = ds.rename(columns={ds.columns[0]: 'Shape'})

color_map1 = {
    'Marble Units': "^",
    'Altered Intrusion': "*",
    'Siliciclastic': "o",
    'Intrusion': "s",
    'Anamolous Rock': "D",
}

ds["Shapes"] = ds["Shape"].map(color_map1)
#dc[dc.columns[0]] = dc[dc.columns[0]].map(color_map)

ds=ds.replace('\<','NaN',regex=True)
ds = ds[np.isfinite(df).all(1)]
ds = ds.dropna()

print(ds)
#print (pca_df_scaled)
print("Shape of y:", ds.shape)
#print("Shape of X:", X.shape)


In [ ]:

# Initialize the 3D graph
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
  
# Define scaled features as arrays
xdata = pca_df_scaled['PC1']
ydata = pca_df_scaled['PC2']
zdata = pca_df_scaled['PC3']

# This will be used to color the scatter plot.
#color_name='Zn_ppm'
#colors = dc['Color']

# Plot 3D scatterplot of PCA
for i in range (len(df)):
    p=ax.scatter3D(
         xdata[i], 
         ydata[i], 
         zdata[i], 
         c= dc['Color'][i],
         marker= ds["Shapes"][i],
    
    #c=zdata, 
    #cmap='Greens', 
    #alpha=0.5
)

#legend
for lithology, color in color_map.items():
    ax.scatter([], [], c=color, label=lithology)
ax.legend(loc=('best'), bbox_to_anchor=(0.15, 0.35), title='Lithology', fontsize='x-small')
plt.show()



# Arrows will be displayed for the following.
show_names=df.columns.tolist()
#['Zn_ppm', 'C_pct', 'S_pct']
indx=df.columns.get_indexer(show_names)
scale=2

# Define the x, y, z variables
loadings = pca.components_
xs = scale*loadings[0]
ys = scale*loadings[1]
zs = scale*loadings[2]

# Plot the loadings
for i , names in enumerate(show_names):
    ip=indx[i]
    #ax.scatter(xs[ip], ys[ip], zs[ip], s=100)
    ax.text(
        #reduced distance from 0.1 to 0.05 for better visualbility
         xs[ip] + 0.05, 
         ys[ip] + 0.05, 
         zs[ip] + 0.05, 
         names)
  
# Plot the arrows
x_arr = np.zeros(len(loadings[0]))
y_arr = z_arr = x_arr
# Plot specific arrow
#indx=df.columns.get_loc("Zn_ppm")

ax.quiver(x_arr[indx], y_arr[indx], z_arr[indx], xs[indx], ys[indx], zs[indx],color='r')

#for i, name in enumerate(df.columns):
    #ip = df.columns.get_loc(name)
    #ax.quiver(x_arr[ip], y_arr[ip], z_arr[ip], xs[ip], ys[ip], zs[ip], color='r')
  
# Plot title of graph
plt.title(f'3D Biplot')
  
# Plot x, y, z labels
ax.set_xlabel('PC1')#, rotation=150)
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')#, rotation=60)
plt.savefig('3D_biplot.svg')


In [ ]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
y = dc['Lithology']
y = y.values.ravel()
X = pca_df_scaled
print("Shape of y:", y.shape)
print("Shape of X:", X.shape)

clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(X, y)

print(clf.predict([[0.3, -0.01, 0.1]]))

In [ ]:
plt.figure(figsize=(10,10))
#plt.scatter(x[:,0],x[:,1],c=data['target'],cmap='plasma')
plt.scatter(x[:,0],x[:,1],c=df['Zn_ppm'],cmap='plasma')

## Plot the arrows
#x_arr = np.zeros(len(loadings[0]))
#y_arr = z_arr = x_arr
#plt.quiver(x_arr, y_arr, z_arr, xs, ys, zs)

plt.xlabel('pc1')
plt.ylabel('pc2')
plt.colorbar()

In [ ]:
# import relevant libraries for 3d graph
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(10,10))
 
# choose projection 3d for creating a 3d graph
axis = fig.add_subplot(111, projection='3d')
 
# x[:,0]is pc1,x[:,1] is pc2 while x[:,2] is pc3
img=axis.scatter(x[:,0],x[:,1],x[:,2], c=df['Zn_ppm'])

# Plot the loadings
for i in range(2):
    axis.scatter(xs[i], ys[i], zs[i], s=200)
    #axis.text(
    #    xs[i] + 0.1, 
    #    ys[i] + 0.1, 
    #    zs[i] + 0.1, 
    #    varnames)
  
# Plot the arrows
x_arr = np.zeros(len(loadings[0]))
y_arr = z_arr = x_arr
axis.quiver(x_arr, y_arr, z_arr, xs, ys, zs)

axis.set_xlabel("PC1")
axis.set_ylabel("PC2")
axis.set_zlabel("PC3")
fig.colorbar(img)

In [ ]:
# Remove all rows that have at least one NaN.
df = df.dropna()

# Remove "<" from the values.
df['Th_ppm'] = df['Th_ppm'].apply(lambda x: x.replace('<', '') if isinstance(x, str) and '<' in x else x)
df['U_ppm'] = df['U_ppm'].apply(lambda x: x.replace('<', '') if isinstance(x, str) and '<' in x else x)

# Change to "object" to "float".
df['Th_ppm'] = df['Th_ppm'].astype(float)
df['U_ppm'] = df['U_ppm'].astype(float)

In [ ]:
# Set the data for clustering
X = df[['Th_ppm', 'U_ppm']].values

# Choose the number of clusters (k)
k = 3

# Initialize and fit the KMeans model
kmeans = KMeans(n_clusters=k)
kmeans.fit(X)

# Add cluster labels to the DataFrame
df['cluster'] = kmeans.labels_

In [ ]:
# Scatter plot with the centroids.
sns.scatterplot(data=df, x="Th_ppm", y="U_ppm", hue=kmeans.labels_)
plt.scatter(kmeans.cluster_centers_[:,0], kmeans.cluster_centers_[:,1], 
            marker="X", c="r", s=80, label="centroids")
plt.legend()
plt.show()

In [ ]:
# Plot the distortion score elbow.
from yellowbrick.cluster import KElbowVisualizer
model = KMeans()
visualizer = KElbowVisualizer(model, k=(1,12)).fit(df)
visualizer.show()

In [ ]:
#import the breast _cancer dataset
from sklearn.datasets import load_breast_cancer
data=load_breast_cancer()
data.keys()
 
# Check the output classes
print(data['target_names'])
 
# Check the input attributes
print(data['feature_names'])

In [ ]:
# construct a dataframe using pandas
df1=pd.DataFrame(data['data'],columns=data['feature_names'])

In [ ]:
df1.head()

In [ ]:
data

In [ ]:
a=[{'T1'},{'T2}]